In [2]:
import pandas as pd
from collections import defaultdict

# --- Define each CSV and its label ---
simulations = {
    "Random": "four_player_tournament_pt2/random_4player_game_logs_pt2.csv",
    "Greedy": "four_player_tournament_pt2/greedy_4player_game_logs_pt2.csv",
    "Mixed": "four_player_tournament_pt2/mixed_4player_game_logs_pt2.csv",
    "FixedOrder": "four_player_tournament_pt2/fixed_order_4player_game_logs_pt2.csv"
}

# --- Analyze each file independently ---
def analyze_file(label, filepath):
    print(f"\n=== {label} Tournament ===")
    df = pd.read_csv(filepath)

    # Get final row of each game
    finals = df.sort_values("round_id").groupby("game_id").tail(1)

    coin_data = []
    win_counts = defaultdict(int)
    games_played = defaultdict(int)

    for _, row in finals.iterrows():
        winners = row["winners"].split(", ")
        for i in range(4):
            bot = row[f"player_{i}_name"]
            coins = row[f"player_{i}_final"]
            coin_data.append((bot, coins))
            games_played[bot] += 1
            if bot in winners:
                win_counts[bot] += 1 / len(winners)  # fractional credit if tie

    # Convert to DataFrame
    coin_df = pd.DataFrame(coin_data, columns=["Bot", "Coins"])
    coin_avg = coin_df.groupby("Bot")["Coins"].mean().round(2)

    # Build summary table
    summary_df = pd.DataFrame({
        "Avg Coins": coin_avg,
        "Games Played": pd.Series(games_played),
        "Win Count": pd.Series(win_counts),
    })
    summary_df["Win %"] = (summary_df["Win Count"] / summary_df["Games Played"] * 100).round(2)
    summary_df = summary_df.sort_values("Win %", ascending=False)

    print(summary_df)
    summary_df.to_csv(f"four_player_tournament_pt2/{label.lower()}_bot_performance_summary.csv")

# --- Run all analyses ---
for label, path in simulations.items():
    analyze_file(label, path)



=== Random Tournament ===
             Avg Coins  Games Played     Win Count  Win %
RandomAgent      12.13         40000  38730.333333  96.83

=== Greedy Tournament ===
             Avg Coins  Games Played   Win Count  Win %
GreedyAgent      23.66           400  395.333333  98.83

=== Mixed Tournament ===
               Avg Coins  Games Played  Win Count  Win %
GreedyAgent        30.73           240      143.0  59.58
RoundBetAgent      26.02           240       72.5  30.21
GameBetAgent       21.00           240       24.5  10.21
MCTSAgent           7.95           240        NaN    NaN

=== FixedOrder Tournament ===
               Avg Coins  Games Played  Win Count  Win %
GreedyAgent        35.60           300      261.5  87.17
RoundBetAgent      22.10           300       33.5  11.17
MCTSAgent           9.83           300        5.0   1.67
GameBetAgent       17.52           300        NaN    NaN
